# Import libraries

In [1]:
import pandas as pd
import numpy as np
from src.common_func import *
from datetime import datetime,timedelta
import time
import sys
sys.path.append('..')

## Import data files, available on data.gov.sg

In [3]:
resale_2012_2014 = pd.read_csv('resale-flat-prices/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv',
                               parse_dates=True)
resale_2015_2016 = pd.read_csv('resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv',
                               parse_dates=True)
resale_2017 = pd.read_csv('resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv',
                               parse_dates=True)

In [4]:
# concat all files together

df = pd.concat([resale_2012_2014,resale_2015_2016,resale_2017],ignore_index = True)
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN


In [5]:
# the floor level is given in X TO Y. Split by 'TO' to get the storey
# thereafter, find the mean

df_storey =df['storey_range'].str.split(pat = " TO ", expand=True)
df_storey = df_storey.astype(float)
df_storeyrange =(df_storey[1] + df_storey[0])/2
df_storeyrange.head()

0    8.0
1    3.0
2    8.0
3    3.0
4    8.0
dtype: float64

In [6]:
df['storey_range'] = df_storeyrange

In [7]:
# drop remaining_lease

features = df.drop(columns = ['remaining_lease'], axis = 1)
features.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,8.0,45.0,Improved,1986,250000.0
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,3.0,44.0,Improved,1980,265000.0
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,8.0,68.0,New Generation,1980,315000.0
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,3.0,67.0,New Generation,1984,320000.0
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,8.0,67.0,New Generation,1980,321000.0


In [8]:
# Get address from block and street name

features['address'] = features['block'] + " " + features['street_name']
features.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,address
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,8.0,45.0,Improved,1986,250000.0,172 ANG MO KIO AVE 4
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,3.0,44.0,Improved,1980,265000.0,510 ANG MO KIO AVE 8
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,8.0,68.0,New Generation,1980,315000.0,610 ANG MO KIO AVE 4
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,3.0,67.0,New Generation,1984,320000.0,474 ANG MO KIO AVE 10
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,8.0,67.0,New Generation,1980,321000.0,604 ANG MO KIO AVE 5


In [9]:
# convert lease commence date to datetime object

features['lease_commence_dt']=pd.to_datetime(features['lease_commence_date'],format="%Y")
features['month'] = pd.to_datetime(features['month'],format="%Y-%m")
features.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,address,lease_commence_dt
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,8.0,45.0,Improved,1986,250000.0,172 ANG MO KIO AVE 4,1986-01-01
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,3.0,44.0,Improved,1980,265000.0,510 ANG MO KIO AVE 8,1980-01-01
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,8.0,68.0,New Generation,1980,315000.0,610 ANG MO KIO AVE 4,1980-01-01
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,3.0,67.0,New Generation,1984,320000.0,474 ANG MO KIO AVE 10,1984-01-01
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,8.0,67.0,New Generation,1980,321000.0,604 ANG MO KIO AVE 5,1980-01-01


In [10]:
# get current lease and remaining lease in terms of year

features['current_lease_dt']=features['month']-features['lease_commence_dt']
features['current_lease_year'] = features['current_lease_dt'] / np.timedelta64(1, 'Y')
features['remaining_lease_year'] = 99-features['current_lease_year']
features.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,address,lease_commence_dt,current_lease_dt,current_lease_year,remaining_lease_year
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,8.0,45.0,Improved,1986,250000.0,172 ANG MO KIO AVE 4,1986-01-01,9556 days,26.163439,72.836561
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,3.0,44.0,Improved,1980,265000.0,510 ANG MO KIO AVE 8,1980-01-01,11748 days,32.164932,66.835068
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,8.0,68.0,New Generation,1980,315000.0,610 ANG MO KIO AVE 4,1980-01-01,11748 days,32.164932,66.835068
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,3.0,67.0,New Generation,1984,320000.0,474 ANG MO KIO AVE 10,1984-01-01,10287 days,28.164849,70.835151
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,8.0,67.0,New Generation,1980,321000.0,604 ANG MO KIO AVE 5,1980-01-01,11748 days,32.164932,66.835068


In [11]:
features = features.drop(columns = ['current_lease_year','current_lease_dt','lease_commence_date']
                         ,axis=1)
features.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,resale_price,address,lease_commence_dt,remaining_lease_year
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,8.0,45.0,Improved,250000.0,172 ANG MO KIO AVE 4,1986-01-01,72.836561
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,3.0,44.0,Improved,265000.0,510 ANG MO KIO AVE 8,1980-01-01,66.835068
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,8.0,68.0,New Generation,315000.0,610 ANG MO KIO AVE 4,1980-01-01,66.835068
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,3.0,67.0,New Generation,320000.0,474 ANG MO KIO AVE 10,1984-01-01,70.835151
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,8.0,67.0,New Generation,321000.0,604 ANG MO KIO AVE 5,1980-01-01,66.835068


In [13]:
# initialise an empty column for slicing

features = features.assign(Coords = np.NaN)
features.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,resale_price,address,lease_commence_dt,remaining_lease_year,Coords
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,8.0,45.0,Improved,250000.0,172 ANG MO KIO AVE 4,1986-01-01,72.836561,NaN
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,3.0,44.0,Improved,265000.0,510 ANG MO KIO AVE 8,1980-01-01,66.835068,NaN
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,8.0,68.0,New Generation,315000.0,610 ANG MO KIO AVE 4,1980-01-01,66.835068,NaN
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,3.0,67.0,New Generation,320000.0,474 ANG MO KIO AVE 10,1984-01-01,70.835151,NaN
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,8.0,67.0,New Generation,321000.0,604 ANG MO KIO AVE 5,1980-01-01,66.835068,NaN


In [14]:
# estimated time just for calculation
len(features)/(250*60)

14.9642

In [ ]:
%%time
# plus 1 to get last part as the quotient*250 will be smaller than the length of the dataframe

x= (len(features) // 250) + 1

for i in range(x):
    # slice dataframe into blocks of 250
    # set min and max to prevent out of bounds error
    a = max(i*250,0)
    b = min((i+1)*250,len(features))

    # control limit to 250 per minute

    start_time = datetime.now()
    plus_1_min = start_time + timedelta(minutes = 1)

    features.iloc[a:b,-1] = features.iloc[a:b]['address'].apply(getcoordinates)
    difference = plus_1_min - datetime.now()

    # if loop takes longer than 1 min, do not wait

    if difference.days < 0:
        continue
    else:
        time.sleep(difference.seconds/60)

# save before cleaning due to the long processing time

features.to_csv('data/geo_coords_data.csv')

In [41]:
features = pd.read_csv('data/geo_coords_data.csv',index_col="Unnamed: 0")

In [42]:
import re
def string_to_array(coords:str):
    split = coords.split(' ')
    list_of_digits = [re.findall("[+-]?[0-9]*[.]*[0-9]+",x) for x in split]
    coords = []
    for i in list_of_digits:
        try:
            float(i[0])
            coords.append(i[0])
        except Exception as e:
            pass
    return coords

In [43]:
features['Coords'] = features['Coords'].apply(string_to_array)

In [44]:
features = pd.merge(features,pd.DataFrame(features['Coords'].tolist()),on=features.index)

In [45]:
features = features.iloc[:,2:]

In [47]:
features.rename(columns = {0:'LAT',1:'LONG'},inplace=True)

In [50]:
features.drop(columns='Coords',inplace=True)

In [51]:
features

,month,town,flat_type,storey_range,floor_area_sqm,flat_model,resale_price,lease_commence_dt,remaining_lease_year,LAT,LONG
0,2012-03-01,ANG MO KIO,2 ROOM,8.0,45.0,Improved,250000.0,1986-01-01,72.836561,1.37469423,103.83646287
1,2012-03-01,ANG MO KIO,2 ROOM,3.0,44.0,Improved,265000.0,1980-01-01,66.835068,1.37340093,103.84907324
2,2012-03-01,ANG MO KIO,3 ROOM,8.0,68.0,New Generation,315000.0,1980-01-01,66.835068,1.3793947,103.83915723
3,2012-03-01,ANG MO KIO,3 ROOM,3.0,67.0,New Generation,320000.0,1984-01-01,70.835151,1.36275785,103.85801532
4,2012-03-01,ANG MO KIO,3 ROOM,8.0,67.0,New Generation,321000.0,1980-01-01,66.835068,1.37986658,103.83597665
...,...,...,...,...,...,...,...,...,...,...,...
224458,2022-09-01,YISHUN,5 ROOM,14.0,113.0,Improved,716000.0,2018-01-01,94.334606,1.41572236,103.84364845
224459,2022-09-01,YISHUN,5 ROOM,8.0,113.0,Improved,636000.0,2016-01-01,92.333196,1.41924161,103.84494165
224460,2022-09-01,YISHUN,5 ROOM,5.0,131.0,Improved,600000.0,1987-01-01,63.333285,1.42612489,103.82751257
224461,2022-09-01,YISHUN,5 ROOM,5.0,122.0,Improved,532000.0,1987-01-01,63.333285,1.42752812,103.82741659


In [52]:
features.to_csv("data/geo_coords_data.csv")